In [47]:
import utils
import skimage
import skimage.morphology
import numpy as np

In [48]:
def fill_holes(im: np.ndarray, starting_points: list, num_iterations: int) -> np.ndarray:
    """
        A function that takes a binary image (im),  and a set of points 
        indicating position of holes, and fills the holes.

        args:
            im: np.ndarray of shape (H, W) with boolean values (dtype=np.bool)
            starting_points: list of list containing starting points (row, col). Ex:
                [[row1, col1], [row2, col2], ...]
            num_iterations: integer defining the number of iterations to apply the 
                            hole filling algorithm
        return:
            (np.ndarray) of shape (H, W). dtype=np.bool
    """
    structuring_element = np.array([
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]
    ], dtype=bool)
    
    # From algorithm 1

    # Input
    I = im.copy()
    I_C = np.invert(im)
    K = num_iterations
    S = starting_points
    B = structuring_element
    
    # Step 1
    X = [np.zeros_like(I, dtype=bool)] # X[0] = array of 0's like Image

    # Step 2
    for row, column in S:
        X[0][row, column] = 1
    
    # Step 3
    for k in range(1, K+1):
        # Take dilation of X_k-1 and B
        dilation = skimage.morphology.binary_dilation(X[k - 1], selem=B)
        # For intersection, take logical_and
        X.append(np.logical_and(dilation, I_C))
    
    # For union, take logical_or
    return np.logical_or(X[K], im)



In [49]:
if __name__ == "__main__":
    im = utils.read_image("balls-with-reflections.png")
    binary_image = im != 0
    starting_points = [ # (row, column)
        [51, 64],
        [44, 180],
        [35, 365],
        [156, 94],
        [141, 264],
        [138, 467],
        [198, 180],
        [229, 413],
        [294, 103],
        [302, 230],
        [368, 388],
        [352, 489],
        [454, 57],
        [457, 236],
        [469, 400],
        [489, 506]
    ]
    num_iterations = 30

    result = fill_holes(binary_image, starting_points, num_iterations)

    assert im.shape == result.shape, "Expected image shape ({}) to be same as resulting image shape ({})".format(
            im.shape, result.shape)
    assert result.dtype == bool, "Expected resulting image dtype to be np.bool. Was: {}".format(
            result.dtype)

    result = utils.to_uint8(result)
    utils.save_im("balls-with-reflections-filled.png", result)

Reading image: img/balls-with-reflections.png
Saving image to: img/processed/balls-with-reflections-filled.png
